In [1]:
import pandas as pd
import joblib   
import numpy as np
from sklearn.model_selection import train_test_split
from SimulateDay import scale_data, predict_action, get_stock_data, add_columns, stock_market_simulation
import xgboost as xgb
import joblib
import altair as alt
onlybuy = pd.read_csv('simResults/only_buy.csv')
newbest = pd.read_csv('simResults/no_mass_trades.csv')
ogsimResults = pd.read_csv('simResults/sim_results copy.csv')
simResults = pd.read_csv('simResults/sim_results.csv')
specific_model_decisions = pd.read_csv('simResults/specific_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])
general_model_decisions = pd.read_csv('simResults/general_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])

In [147]:
general_model_decisions[general_model_decisions['Action'] == 'Buy']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
209,ABT,4,Buy,116.550003,9883.449997,1,10000.000000,2024-10-15
180,ACN,0,Buy,365.079987,9634.920013,1,10000.000000,2024-10-09
169,ADBE,4,Buy,508.570007,9491.429993,1,10000.000000,2024-10-15
1486,ADM,1,Buy,57.290001,9942.709999,1,10000.000000,2024-10-10
1488,ADM,3,Buy,57.910000,9884.799999,2,10000.619999,2024-10-14
...,...,...,...,...,...,...,...,...
1388,XYL,3,Buy,137.240005,9860.639999,1,9997.880005,2024-10-14
1259,YUM,4,Buy,134.149994,9865.850006,1,10000.000000,2024-10-15
1749,ZBH,4,Buy,105.150002,9894.849998,1,10000.000000,2024-10-15
1924,ZBRA,4,Buy,374.739990,9625.260010,1,10000.000000,2024-10-15


In [148]:
specific_model_decisions[specific_model_decisions['Action']  == 'Buy']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
5,AAPL,0,Buy,229.539993,9770.460007,1.0,10000.000000,2024-10-09
6,AAPL,1,Buy,229.039993,9541.420013,2.0,9999.500000,2024-10-10
22,ACGL,1,Buy,113.900002,9886.099998,1.0,10000.000000,2024-10-10
26,ACN,0,Buy,365.079987,9634.920013,1.0,10000.000000,2024-10-09
30,ACN,4,Buy,368.660004,9266.260010,2.0,10003.580017,2024-10-15
...,...,...,...,...,...,...,...,...
2453,XOM,4,Buy,120.349998,9757.560005,2.0,9998.260002,2024-10-15
2494,XOM,5,Buy,121.089996,9756.820007,2.0,9999.000000,2024-10-16
2465,ZBH,1,Buy,102.820000,9897.180000,1.0,10000.000000,2024-10-10
2468,ZBH,4,Buy,104.599998,9895.400002,1.0,10000.000000,2024-10-15


In [151]:
specific_model_decisions[(specific_model_decisions['Portfolio Value'] > 10000) & (specific_model_decisions['Day'] == 5)]

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
2479,AAPL,5,Sell,231.097595,10001.557602,0.0,10001.557602,2024-10-16
2515,ACN,5,Buy,372.225006,9262.695007,2.0,10007.145020,2024-10-16
2661,AEP,5,Buy,101.279999,9801.000000,2.0,10003.559998,2024-10-16
2906,AKAM,5,Buy,103.339996,9793.550003,2.0,10000.229996,2024-10-16
2688,AMP,5,Buy,513.729980,8993.030029,2.0,10020.489990,2024-10-16
...,...,...,...,...,...,...,...,...
2950,TPR,5,Hold,45.250000,9955.189999,1.0,10000.439999,2024-10-16
2765,TSCO,5,Buy,301.829895,9397.210114,2.0,10000.869904,2024-10-16
2896,TXT,5,Buy,88.120003,9825.540001,2.0,10001.780006,2024-10-16
2529,WFC,5,Buy,63.775002,9878.684998,2.0,10006.235001,2024-10-16


In [71]:
specific_model_decisions.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value
count,2479.000000,2479.000000,2479.000000,2479.000000,2479.000000
mean,2.208148,221.707100,9888.981120,0.478869,10000.354979
std,1.725584,523.571315,540.387454,0.814211,5.835473
min,0.000000,7.520000,0.000000,0.000000,9857.950195
25%,1.000000,69.195000,9919.555000,0.000000,10000.000000
50%,2.000000,124.260002,10000.000000,0.000000,10000.000000
75%,3.000000,232.174995,10000.000000,1.000000,10000.000000
max,5.000000,9768.290039,10015.770020,4.000000,10140.690430


In [81]:
general_model_decisions.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value
count,2398.000000,2398.000000,2398.000000,2398.000000,2398.000000
mean,1.997915,229.844580,9938.495348,0.310259,9999.914294
std,1.414949,532.446335,187.913517,0.596600,1.761543
min,0.000000,7.520000,7189.040039,0.000000,9981.630005
25%,1.000000,71.060001,9992.270000,0.000000,10000.000000
50%,2.000000,132.224998,10000.000000,0.000000,10000.000000
75%,3.000000,238.057503,10000.000000,0.000000,10000.000000
max,4.000000,9842.889648,10000.979996,4.000000,10021.869995


In [200]:
modelComparison = pd.read_csv('simResults/MODELCOMPARISON.csv', index_col=0)
modelComparison.sort_values(by=['Total Profit %'], ascending=False).head(10)


,Model,Stock,Final Portfolio Value,Total Profit %
5,Stacking,AAPL,12560.38,25.60
1,Random Forest,AAPL,12438.13,24.38
3,XGBoost,AAPL,12404.69,24.04
4,Specific Model,AAPL,12404.69,24.04
0,Logistic Regression,AAPL,12010.10,20.01
9,XGBoost,MSFT,11808.35,18.08
8,SVM,MSFT,11803.34,18.03
10,Specific Model,MSFT,11774.20,17.74
6,Logistic Regression,MSFT,11677.37,16.77
7,Random Forest,MSFT,11187.02,11.87


In [201]:
import altair as alt

chart = alt.Chart(modelComparison).mark_bar().encode(
    x=alt.X('Stock:N', title='Tested Stocks'),
    y=alt.Y('Total Profit %:Q', title='Total Profit %'),
    color='Model:N',
    column='Model:O'
).properties(
    width=100,
    height=800,
    title='Total Profit % by Model Over Year To date Simulation',
    center=True
).configure_axis(
    labelAngle=45
)

chart.display()


alt.Chart(...)

In [129]:
def get_final_portfolio_values(df):
    # Group by 'Stock Name' and get the last row for each group
    final_values = df.groupby('Stock Name').apply(lambda x: x.iloc[-1])
    
    # Extract 'Stock Name' and 'Portfolio Value' columns
    result = final_values[['Stock Name', 'Portfolio Value','Shares Held']].reset_index(drop=True)
    
    return result


In [131]:
final_portfolio_values1 = get_final_portfolio_values(newbest)#.drop(simResults[simResults['Stock Name']=='NFLX'].index))
final_portfolio_values1['Profit %'] = (final_portfolio_values1['Portfolio Value'] - 10000) / 10000 * 100
alt.Chart(final_portfolio_values1).mark_bar().encode(
    x='Stock Name',
    y='Profit %',
    color=alt.condition(
        alt.datum['Profit %'] > 0,
        alt.value('green'),
        alt.value('red')
    )
).properties(
    title='Final Portfolio Value by Stock',
    width=800,
    height=400
).configure_axis(
    labelAngle=45
).display()

alt.Chart(...)

In [139]:
final_portfolio_values0 = get_final_portfolio_values(onlybuy)#.drop(simResults[simResults['Stock Name']=='NFLX'].index))
final_portfolio_values0['Profit %'] = (final_portfolio_values0['Portfolio Value'] - 10000) / 10000 * 100
alt.Chart(final_portfolio_values0).mark_bar().encode(
    x='Stock Name',
    y='Profit %',
    color=alt.condition(
        alt.datum['Profit %'] > 0,
        alt.value('green'),
        alt.value('red')
    )
).properties(
    title='Final Portfolio Value by Stock',
    width=800,
    height=400
).configure_axis(
    labelAngle=45
).display()

alt.Chart(...)

In [120]:
final_portfolio_values = get_final_portfolio_values(simResults)#.drop(simResults[simResults['Stock Name']=='NFLX'].index))
final_portfolio_values['Profit %'] = (final_portfolio_values['Portfolio Value'] - 10000) / 10000 * 100
alt.Chart(final_portfolio_values).mark_bar().encode(
    x='Stock Name',
    y='Profit %',
    color=alt.condition(
        alt.datum['Profit %'] > 0,
        alt.value('green'),
        alt.value('red')
    )
).properties(
    title='Final Portfolio Value by Stock',
    width=800,
    height=400
).configure_axis(
    labelAngle=45
).display()

alt.Chart(...)

In [140]:
final_portfolio_values.describe() , final_portfolio_values1.describe(), final_portfolio_values0.describe()

(       Portfolio Value  Shares Held   Profit %
 count        21.000000    21.000000  21.000000
 mean      10891.237419    18.016460   8.912374
 std        2343.959668    34.993873  23.439597
 min        8714.090503    -4.850322 -12.859095
 25%        9927.830093     0.000000  -0.721699
 50%        9986.659996     5.000000  -0.133400
 75%       10897.731785    18.295126   8.977318
 max       19849.520512   150.832216  98.495205,
        Portfolio Value  Shares Held   Profit %
 count        21.000000    21.000000  21.000000
 mean      10706.286930    17.727816   7.062869
 std        1268.521332    22.526169  12.685213
 min        9440.909296    -0.300420  -5.590907
 25%        9943.790085     2.000000  -0.562099
 50%       10001.640001    10.000000   0.016400
 75%       10848.622707    18.523928   8.486227
 max       14057.461449    77.000000  40.574614,
        Portfolio Value  Shares Held    Profit %
 count        21.000000    21.000000   21.000000
 mean      11420.838853    26.452124

In [96]:
import plotly.subplots as sp
import plotly.graph_objects as go

def plot_portfolio_value_and_close_price(df, symbol):
    df = df[df['Stock Name'] == symbol]
    fig = sp.make_subplots(rows=2, cols=1)

    fig.add_trace(
        go.Scatter(
            x=df['Day'],
            y=df['Portfolio Value'],
            mode='lines',
            name='Portfolio Value'
        )
    )

    fig.add_trace(go.Scatter(
            x=df['Day'],
            y=df['Stock Price'],
            mode='lines',
            name='Stock Price'
        ), row=2, col=1
    )

    fig.add_trace(go.Bar(
            x=df['Day'],
            y=df['Shares Held'],
            name='Shares Held'
        ), row=2, col=1
    )

    fig.update_layout(
        title=f'Portfolio Value and Close Price for {symbol}',
        xaxis_title='Day',
        yaxis_title='Value',
        hovermode='x unified', # Compare data points on hover

    )

    fig.show()


In [100]:

plot_portfolio_value_and_close_price(simResults, 'GOOGL')

In [134]:
simResults['Profit %'] = (simResults['Portfolio Value'] - 10000) / 10000 * 100
newbest['Profit %'] = (newbest['Portfolio Value'] - 10000) / 10000 * 100
simResults.describe(), newbest.describe()

(               Day  Stock Price          Cash  Shares Held  Portfolio Value  \
 count  7665.000000  7665.000000   7665.000000  7665.000000      7665.000000   
 mean    182.000000   191.353528   7778.772554    14.289348     10407.219051   
 std     105.372903   149.867115   3291.011870    24.505178      1242.552877   
 min       0.000000    12.450000      0.000000    -4.949396      8256.222256   
 25%      91.000000    67.150002   6741.840057     0.000000      9977.639999   
 50%     182.000000   167.660004   9518.069988     4.000000     10004.069998   
 75%     273.000000   251.119995   9978.153339    19.000000     10319.739868   
 max     364.000000   730.289978  14052.810135   153.494969     20810.848167   
 
           Profit %  
 count  7665.000000  
 mean      4.072191  
 std      12.425529  
 min     -17.437777  
 25%      -0.223600  
 50%       0.040700  
 75%       3.197399  
 max     108.108482  ,
                Day  Stock Price          Cash  Shares Held  Portfolio Value  \

In [118]:
final_portfolio_values.describe()

,Portfolio Value,Shares Held,Profit %
count,21.000000,21.000000,21.000000
mean,10891.237419,18.016460,8.912374
std,2343.959668,34.993873,23.439597
min,8714.090503,-4.850322,-12.859095
25%,9927.830093,0.000000,-0.721699
50%,9986.659996,5.000000,-0.133400
75%,10897.731785,18.295126,8.977318
max,19849.520512,150.832216,98.495205


In [42]:
simResults

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
0,AAPL,0,Buy,167.449997,9832.550003,1.000000,10000.000000,2023-05-03
1,AAPL,1,Buy,165.789993,9666.760010,2.000000,9998.339996,2023-05-04
2,AAPL,2,Hold,173.570007,9666.760010,2.000000,10013.900024,2023-05-05
3,AAPL,3,Buy,173.500000,9493.260010,3.000000,10013.760010,2023-05-08
4,AAPL,4,Buy,171.770004,9321.490005,4.000000,10008.570023,2023-05-09
...,...,...,...,...,...,...,...,...
7660,UNH,360,Buy,581.580017,0.000000,18.146646,10553.726711,2024-10-08
7661,UNH,361,Buy,591.219971,0.000000,18.146646,10728.659537,2024-10-09
7662,UNH,362,Hold,597.700012,0.000000,18.146646,10846.250557,2024-10-10
7663,UNH,363,Buy,598.049988,0.000000,18.146646,10852.601440,2024-10-11
